In [2]:
import os
import sys
import glob
import gzip
import psutil
import shutil
from collections import OrderedDict, defaultdict, Counter
import multiprocessing as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pysam
from pyBioInfo.Range import GRange
from pyBioInfo.IO.File import BamFile, FastaFile, GtfFile, GtfTranscriptBuilder, BedFile
from pyBioInfo.Utils import BlockTools, ShiftLoader, SegmentTools

In [48]:
infile = "data/NanoNASCseq_All.xlsx"
outfile = "data/NanoNASCseq.xlsx"

d = pd.read_excel(infile)
d = d[d["Run"] != "20220130_Cell8"] # R9.4
d = d[d["Run"] != "20220316_Cell40"]
d = d[d["Run"] != "20220413_Cell26"]
d = d[d["Run"] != "20220506_Cell25"]
d = d[d["Run"] != "20220520_Cell16"]
d = d[d["Run"] != "20220525_Run1"]
d = d[d["Run"] != "20220525_Run2"]
d = d[d["Run"] != "20220601_Cell22"]
d = d[d["Run"] != "20220607_Cell40"]
d = d[d["Run"] != "20220615_Cell68"]
d = d[d["Run"] != "20220621_R10Test"] # Library structure

d = d[d["Run"] != "20220719_Embryo"] # Without blastocyst cells
d = d[d["Run"] != "20220729_Embryo"] # R9.4
d = d[d["Run"] != "20221019_Blastocyst"] # Bad chip
d = d[d["Run"] != "20220912_HumanBlastR1"]
d = d[d["Run"] != "20220912_HumanBlastR2"]

d = d[d["Run"] != "20220818_mESCR3"] # R10.4.1
d = d[d["Run"] != "20220818_mESCR3M"] # Merged
d = d[d["Run"] != "20220902_Blastocyst"]
d = d[d["Run"] != "20220902_BlastocystM"]
d = d[d["Run"] != "20220903_Blastocyst"]
d = d[d["Run"] != "20220903_BlastocystM"]

d.to_excel(outfile, index=False)
print(len(d))

4555


# 1. Report summary of datasets

In [52]:
discarded_runs = [
    # R9.4 and test library structure
    "20220130_Cell8",
    "20220316_Cell40",
    "20220413_Cell26",
    "20220506_Cell25",
    "20220520_Cell16",
    "20220525_Run1",
    "20220525_Run2",
    "20220601_Cell22",
    "20220607_Cell40",
    "20220615_Cell68",

    # test library structure
    "20220621_R10Test",

    # without blastocyst cell
    "20220719_Embryo",

    # R9.4
    "20220729_Embryo",

    # bad chip
    "20221019_Blastocyst",

    # human blastocyst
    "20220912_HumanBlastR1",
    "20220912_HumanBlastR2",

    # others
    "20220818_mESCR3",
    "20220818_mESCR3M",
    "20220902_Blastocyst",
    "20220902_BlastocystM",
    "20220903_Blastocyst",
    "20220903_BlastocystM"
]

## 1. make data/NanoNASCseq.xlsx

In [55]:
d = pd.read_excel("data/NanoNASCseq_All.xlsx")
print("Input cells:", len(d))
for run in discarded_runs:
    d = d[d["Run"] != run]
d.to_excel("data/NanoNASCseq.xlsx", index=False)
print("Output cells:", len(d))

Input cells: 5569
Output cells: 4555


## 2. make data/NanoNASCseq_All_Summary.xlsx

In [56]:
def divide(a, b):
    return None if (b is None or b == 0) else a / b
dat = pd.read_excel("data/NanoNASCseq_All.xlsx")

In [57]:
# Cell reads and trimmed reads
cell_reads_list = []
trim_reads_list = []
for run, cell in dat[["Run", "Cell"]].values:
    cell_reads = np.nan
    trim_reads = np.nan
    path = "results/demux/trimmed/%s/%s/stats.tsv" % (run, cell)
    if os.path.exists(path):
        df = pd.read_csv(path, sep="\t")
        cell_reads = df["Total"].values[0]
        trim_reads = df["Pass"].values[0]
    cell_reads_list.append(cell_reads)
    trim_reads_list.append(trim_reads)
dat["Cell.Reads"] = cell_reads_list
dat["Trimmed.Reads"] = trim_reads_list
dat["Trimmed.Ratio"] = dat["Trimmed.Reads"] / dat["Cell.Reads"]

In [58]:
# Mapped reads
mapped_reads_list = []
for run, cell in dat[["Run", "Cell"]].values:
    mapped_reads = np.nan
    path = "results/mapping/minimap2/%s/%s.flagstat" % (run, cell)
    if os.path.exists(path):
        mapped_reads = int(open(path).readlines()[7].split()[0])
    mapped_reads_list.append(mapped_reads)
dat["Mapped.Reads"] = mapped_reads_list
dat["Mapped.Ratio"] = dat["Mapped.Reads"] / dat["Trimmed.Reads"]

In [59]:
# Mitochondrion reads ratio
mito_ratio_list = []
for run, cell in dat[["Run", "Cell"]].values:
    mito_ratio = np.nan
    path = "results/mapping/chrom_reads/%s/%s.tsv" % (run, cell)
    if os.path.exists(path):
        try:
            v1, v2 = 0, 0
            for line in open(path):
                chrom, count = line.strip("\n").split("\t")
                count = int(count)
                v1 += count
                if chrom == "chrM":
                    v2 = count
            mito_ratio = divide(v2, v1)
        except ValueError:
            print(path)
    mito_ratio_list.append(mito_ratio)
dat["Mito.Ratio"] = mito_ratio_list

In [60]:
# Filtered reads
filtered_reads_list = []
filtered_clip_reads_list = []
for run, cell in dat[["Run", "Cell"]].values:
    filtered_reads = np.nan
    filtered_clip_reads = np.nan
    path = "results/mapping/stat_clip/%s/%s.log" % (run, cell)
    if os.path.exists(path):
        lines = open(path).readlines()
        for i, line in enumerate(lines):
            if line.startswith("Input"):
                v1, v2 = lines[i + 1].split("\t")[:2]
                filtered_reads = int(v1)
                filtered_clip_reads = int(v2)
                break
    filtered_reads_list.append(filtered_reads)
    filtered_clip_reads_list.append(filtered_clip_reads)
dat["Filtered.Reads"] = filtered_reads_list
dat["Filtered.Ratio"] = dat["Filtered.Reads"] / dat["Mapped.Reads"]
dat["FilteredClip.Reads"] = filtered_clip_reads_list
dat["FilteredClip.Ratio"] = dat["FilteredClip.Reads"] / dat["Filtered.Reads"]

In [61]:
# Duplicate reads
umis1_list = []
umis2_list = []
for run, cell in dat[["Run", "Cell"]].values:
    path = "results/mapping/mark_duplicate/%s/%s.tsv" % (run, cell)
    umis1 = np.nan
    umis2 = np.nan
    if os.path.exists(path):
        d1 = pd.read_csv(path, sep="\t", header=0)
        umis1 = len(d1)
        umis2 = len(d1[d1["AllSize"] >= 2])
    umis1_list.append(umis1)
    umis2_list.append(umis2)
dat["UMIs"] = umis1_list
dat["UMIs.2Reads"] = umis2_list
dat["Duplicate.Reads"] = dat["FilteredClip.Reads"] - dat["UMIs"]
dat["Duplicate.Ratio"] = dat["Duplicate.Reads"] / dat["FilteredClip.Reads"]
dat["Unique.Reads"] = dat["UMIs"]

In [62]:
# Detected genes
genes_list = []
for run, cell in dat[["Run", "Cell"]].values:
    genes = np.nan
    path = "results/expression/quant_genes/min_read_1_min_tc_1/%s/%s.tsv" % (run, cell)
    if os.path.exists(path):
        d = pd.read_csv(path, sep="\t", header=0, index_col=0)
        d = d[d["Total"] > 0]
        genes = len(set(filter(lambda x: "_" not in x, d.index)))
    genes_list.append(genes)
dat["Genes"] = genes_list

In [63]:
# Detected isoforms
sc_list = set(['full-splice_match', 'incomplete-splice_match', 'novel_in_catalog', 'novel_not_in_catalog'])
isoforms1_list = []
isoforms2_list = []
for run, cell in dat[["Run", "Cell"]].values:
    isoforms1 = np.nan # number of assemblied isoforms
    isoforms2 = np.nan # number of known isoforms
    path = "results/assembly/sqanti3/%s/%s/%s_classification.txt" % (run, cell, cell)
    if os.path.exists(path):
        d = pd.read_csv(path, sep="\t", header=0, index_col=0)
        isoforms1 = len(d)
        d = d[[sc in sc_list for sc in d["structural_category"]]]
        isoforms2 = len(d)
    isoforms1_list.append(isoforms1)
    isoforms2_list.append(isoforms2)
dat["Isoforms.Assembled"] = isoforms1_list
dat["Isoforms.Known"] = isoforms2_list

In [64]:
# Mismatch ratios
mtypes = []
for base1 in "ACGT":
    for base2 in "ACGT":
        if base1 != base2:
            mtypes.append("%s%s" % (base1, base2))
ratios = dict()
for mtype in mtypes:
    ratios[mtype] = list()
for run, cell in dat[["Run", "Cell"]].values:
    path = "results/mismatch/ratio_consensus/%s/%s.tsv" % (run, cell)
    if os.path.exists(path):
        df = pd.read_csv(path, sep="\t", index_col=0)
        for mtype in mtypes:
            ratios[mtype].append(df.loc[mtype]["Ratio"])
    else:
        for mtype in mtypes:
            ratios[mtype].append(np.nan)
for mtype in mtypes:
    dat["%s.Ratio" % mtype] = ratios[mtype]

In [65]:
# Pe and Pc
pe_list = []
pc_list = []
for run, cell in dat[["Run", "Cell"]].values:
    pe = np.nan
    pc = np.nan
    path = "results/signal2noise/pc/%s/%s.tsv" % (run, cell)
    if os.path.exists(path):
        d = pd.read_csv(path, sep="\t")
        pe, pc, pc_pe = d.iloc[0]
    pe_list.append(pe)    
    pc_list.append(pc)
dat["Pe"] = pe_list
dat["Pc"] = pc_list
dat["PcPe.Ratio"] = dat["Pc"] / dat["Pe"]

In [66]:
# Nascent UMIs
nascent_umis_list = []
for run, cell in dat[["Run", "Cell"]].values:
    nascent_umis = np.nan
    path = "results/mismatch/ratio_consensus/%s/%s.events.tsv" % (run, cell)
    if os.path.exists(path):
        d = pd.read_csv(path, sep="\t", index_col=0)
        d = d[(d["Size"] >= 2) & (d["T-C"] >= 2)]
        nascent_umis = len(d)
    nascent_umis_list.append(nascent_umis)
dat["UMIs.2Reads.Nascent.2TCs"] = nascent_umis_list
dat["UMIs.2Reads.Nascent.2TCs.Ratio"] = dat["UMIs.2Reads.Nascent.2TCs"] / dat["UMIs.2Reads"]

In [67]:
# Nascent gene number
genes_list = []
nascent_genes_list = []
for run, cell in dat[["Run", "Cell"]].values:
    genes = np.nan
    nascent_genes = np.nan
    path = "results/expression/quant_genes/min_read_2_min_tc_2/%s/%s.tsv" % (run, cell)
    if os.path.exists(path):
        d = pd.read_csv(path, sep="\t", header=0, index_col=0)
        d = d[d["Total"] > 0]
        genes = len(set(filter(lambda x: "_" not in x, d.index)))
        d = d[d["Nascent"] > 0]
        nascent_genes = len(set(filter(lambda x: "_" not in x, d.index)))
    genes_list.append(genes)
    nascent_genes_list.append(nascent_genes)
dat["Genes.2Reads"] = genes_list
dat["Genes.2Reads.Nascent.2TCs"] = nascent_genes_list

In [68]:
dat.to_excel("data/NanoNASCseq_All_Summary.xlsx", index=False)

## 3. Make data/NanoNASCseq_Summary.xlsx

In [69]:
d = pd.read_excel("data/NanoNASCseq_All_Summary.xlsx")
print("Input cells:", len(d))
for run in discarded_runs:
    d = d[d["Run"] != run]
d.to_excel("data/NanoNASCseq_Summary.xlsx", index=False)
print("Output cells:", len(d))

Input cells: 5569
Output cells: 4555


In [1]:
d = pd.read_excel("NanoNASCseq_Summary.xlsx")
# d = d[np.isnan(d["ActD"])]

df1 = d[d["Strain"] == "K562"]
df2 = d[d["Strain"] == "mESC"]
strains = ['Blastocyst', 
           'Early-blastocyst', 'Early-blastocystX', 
           'Late-blastocyst', 'Late-blastocystX', 
           'Mid-blastocyst', 'Mid-blastocystX']
df3 = d[[x in strains for x in d["Strain"]]]
print("K562:", len(df1))
print("mESC:", len(df2))
print("Blastocyst:", len(df3))

for df in [df1, df2, df3]:
    print()
    print("-" * 40)
    print("4sU\tTime\tCells\tActD(-)\tActD(+)")
    print("-" * 40)
    for s4u, d1 in df.groupby(by="s4U"):
        for t, d2 in d1.groupby(by="Time"):
            n1 = sum(np.isnan(d2["ActD"]))
            n2 = sum(~np.isnan(d2["ActD"]))
            print(s4u, t, len(d2), n1, n2, sep="\t")

NameError: name 'pd' is not defined

In [19]:
d = pd.read_excel("NanoNASCseq_summary_selected_qc.xls")
# d = d[np.isnan(d["ActD"])]

df1 = d[d["Strain"] == "K562"]
df2 = d[d["Strain"] == "mESC"]
strains = ['Blastocyst', 
           'Early-blastocyst', 'Early-blastocystX', 
           'Late-blastocyst', 'Late-blastocystX', 
           'Mid-blastocyst', 'Mid-blastocystX']
df3 = d[[x in strains for x in d["Strain"]]]
print("K562:", len(df1))
print("mESC:", len(df2))
print("Blastocyst:", len(df3))

for df in [df1, df2, df3]:
    print()
    print("-" * 40)
    print("4sU\tTime\tCells\tActD(-)\tActD(+)")
    print("-" * 40)
    for s4u, d1 in df.groupby(by="s4U"):
        for t, d2 in d1.groupby(by="Time"):
            n1 = sum(np.isnan(d2["ActD"]))
            n2 = sum(~np.isnan(d2["ActD"]))
            print(s4u, t, len(d2), n1, n2, sep="\t")

K562: 684
mESC: 216
Blastocyst: 1922

----------------------------------------
4sU	Time	Cells	ActD(-)	ActD(+)
----------------------------------------
0	3.0	293	168	125
50	0.25	28	28	0
50	0.5	47	47	0
50	1.0	34	34	0
50	2.0	29	29	0
50	3.0	229	116	113
100	3.0	12	12	0
200	3.0	2	2	0
400	3.0	1	1	0
500	3.0	9	9	0

----------------------------------------
4sU	Time	Cells	ActD(-)	ActD(+)
----------------------------------------
0	3.0	61	61	0
50	3.0	65	65	0
400	3.0	90	90	0

----------------------------------------
4sU	Time	Cells	ActD(-)	ActD(+)
----------------------------------------
0	3.0	124	114	10
100	3.0	8	8	0
200	3.0	33	33	0
300	3.0	26	26	0
400	3.0	1671	1609	62
500	3.0	33	33	0
600	3.0	27	27	0


# Calculate half-life (deprecated)

    K562, mESC

In [ ]:
info = pd.read_excel("data/NanoNASCseq_Summary.xlsx")

In [47]:
params = [
    ["K562", 50, 0.005, "/home/chenzonggui/species/homo_sapiens/GRCh38.p13/gencode.v39.genes.tsv"],
    ["mESC", 400, 0.008, "/home/chenzonggui/species/mus_musculus/GRCm38.p6/gencode.vM25.genes.tsv"]
]

for cell_line, s4u, min_tc_ratio, gene_anno_path in params:
    d = info
    d = d[d["UMIs"] >= 5000]
    # d = d[d["Genes"] >= 2000]
    d = d[d["CellLine"] == cell_line]
    d = d[d["TC.Ratio"] >= min_tc_ratio]
    d = d[d["Time"] == 3]
    d = d[d["s4U"] == s4u]
    d = d[d["ActD"].isna()]
    
    for tc in [1, 2]:
        print("-" * 80)
        print("Cell line:", cell_line)
        print("Gene annotation:", gene_anno_path)
        print("Min TC count:", tc)
        print("Cells:", len(d))
        

--------------------------------------------------------------------------------
Cell line: K562
Gene annotation: /home/chenzonggui/species/homo_sapiens/GRCh38.p13/gencode.v39.genes.tsv
Min TC count: 1
Cells: 151
--------------------------------------------------------------------------------
Cell line: K562
Gene annotation: /home/chenzonggui/species/homo_sapiens/GRCh38.p13/gencode.v39.genes.tsv
Min TC count: 2
Cells: 151
--------------------------------------------------------------------------------
Cell line: mESC
Gene annotation: /home/chenzonggui/species/mus_musculus/GRCm38.p6/gencode.vM25.genes.tsv
Min TC count: 1
Cells: 110
--------------------------------------------------------------------------------
Cell line: mESC
Gene annotation: /home/chenzonggui/species/mus_musculus/GRCm38.p6/gencode.vM25.genes.tsv
Min TC count: 2
Cells: 110


In [20]:
strains = ["K562", "mESC"]

for strain in strains:
    
    # parameters
    
    if strain == "K562":
        s4u = 50
        gene_anno_path = "/home/chenzonggui/species/homo_sapiens/GRCh38.p13/gencode.v39.genes.tsv"
    elif strain == "mESC":
        s4u = 400
        gene_anno_path = "/home/chenzonggui/species/mus_musculus/GRCm38.p6/gencode.vM25.genes.tsv"
    else:
        assert False
    
    # half-life
    
    for min_tc in [1, 2]:
        dat = pd.read_excel("NanoNASCseq_summary_selected_qc.xls")
        d = dat[(dat["Strain"] == strain) & (dat["Time"] == 3) & (dat["s4U"] == s4u) & (np.isnan(dat["ActD"]))]
        
        print("-" * 80)
        print("Strain:", strain)
        print("Annotation:", gene_anno_path)
        print("Min TC count:", min_tc)
        print("Selected cells:", len(d))

        anno = pd.read_csv(gene_anno_path, sep="\t", index_col=2)
        array1 = []
        array2 = []
        for run, cell in d[["Run", "Cell"]].values:
            path = "results/expression/quantify/%dTC/%s/%s/quant_gene.tsv.gz" % (min_tc, run, cell)
            df = pd.read_csv(gzip.open(path), sep="\t", index_col=0)
            s1 = df["Total"]
            s2 = df["Nascent"]
            s1.name = cell
            s2.name = cell
            array1.append(s1)
            array2.append(s2)
        df1 = pd.concat(array1, axis=1, sort=False).fillna(0)
        df2 = pd.concat(array2, axis=1, sort=False).fillna(0)
        s1 = df1.sum(axis=1)
        s2 = df2.sum(axis=1)
        s1.name = "Total"
        s2.name = "Nascent"
        
        df = pd.concat([s1, s2], axis=1, sort=False).fillna(0)
        df.index.name = "GeneID"
        df["TPM"] = df["Total"] * 1e6 / sum(df["Total"])
        df["A0"] = df["TPM"]
        df["NTR"] = df["Nascent"] / df["Total"]
        df["T"] = -3 / np.log2(1 - df["NTR"])
        df["K"] = df["A0"] * np.log(2) / df["T"]
        # print(set(df.index) - set(anno.index))
        df = df.merge(anno, left_index=True, right_index=True, how="right")
        df.to_csv("results/halflife/NanoNASCseq_%s_%duM_3h_%dcells.%dTC.tsv" % (strain, s4u, len(d), min_tc), sep="\t")

--------------------------------------------------------------------------------
Strain: K562
Annotation: /home/chenzonggui/species/homo_sapiens/GRCh38.p13/gencode.v39.genes.tsv
Min TC count: 1
Selected cells: 116


/home/chenzonggui/miniconda3/envs/py36/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


--------------------------------------------------------------------------------
Strain: K562
Annotation: /home/chenzonggui/species/homo_sapiens/GRCh38.p13/gencode.v39.genes.tsv
Min TC count: 2
Selected cells: 116


/home/chenzonggui/miniconda3/envs/py36/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


--------------------------------------------------------------------------------
Strain: mESC
Annotation: /home/chenzonggui/species/mus_musculus/GRCm38.p6/gencode.vM25.genes.tsv
Min TC count: 1
Selected cells: 90


/home/chenzonggui/miniconda3/envs/py36/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


--------------------------------------------------------------------------------
Strain: mESC
Annotation: /home/chenzonggui/species/mus_musculus/GRCm38.p6/gencode.vM25.genes.tsv
Min TC count: 2
Selected cells: 90


/home/chenzonggui/miniconda3/envs/py36/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)
